# Metoda Sita Liczbowego
Zapoznaj się z dołączonym do notebooka materiałem dydaktycznym, w którym wytłumaczone zostało działanie Metody Sita Liczbowego.


## Zadanie 1. Liczby półpierwsze
Liczby półpierwsze to liczby naturalne będące iloczynem dokładnie 2, niekoniecznie różnych liczb pierwszych np. 33 = 3\*11. Zaimplementuj 2 funkcje (oraz funkcje pomocnicze jeśli są konieczne): generator bazy czynników pierwszych dla danego B (liczby pierwsze nie mogą być większe od B) oraz funkcję NWD. Za ich pomocą sprawdź, które liczby z tablicy są liczbami półpierwszymi.


In [1]:
import math
numbers = [23, 47, 85, 92, 123, 144, 217, 393, 403, 447]


# Przykładowe rozwiązanie

def check_prime(n): # Test pierwszości
    if n < 2:
        return False

    for i in range(2, int(math.sqrt(n))+1):
        if n % i == 0:
            return False
    return True

def gcd(a, b): # Funkcja NWD (algorytm Euklidesa)- zwraca największy wspólny dzielnik a i b
    while b:
        a, b = b, a%b
    return a

        
def factor_base(B): # Zwraca listę zawierającą l. pierwsze nie większe od B
    fbase = []

    for i in range(2, B+1):
        if check_prime(i):
            fbase.append(i)
    
    return fbase

fbase = factor_base(149)

for num in numbers:
    for prime in fbase:
        
        dzielnik = gcd(num, prime)
        semiprime = 0
        
        if gcd(num, prime) == prime and num / prime in fbase:
            print(f"Liczba {num} jest półpierwsza, {num} = {prime}*{num // prime}")
            semiprime = 1
            break
    
    if not semiprime:
        print(f"Liczba {num} nie jest półpierwsza")


Liczba 23 nie jest półpierwsza
Liczba 47 nie jest półpierwsza
Liczba 85 jest półpierwsza, 85 = 5*17
Liczba 92 nie jest półpierwsza
Liczba 123 jest półpierwsza, 123 = 3*41
Liczba 144 nie jest półpierwsza
Liczba 217 jest półpierwsza, 217 = 7*31
Liczba 393 jest półpierwsza, 393 = 3*131
Liczba 403 jest półpierwsza, 403 = 13*31
Liczba 447 jest półpierwsza, 447 = 3*149


## Zadanie 2. Liczby gładkie
Sprawdź czy podane liczby są liczbami B-gładkimi dla danego B, a jeśli tak to wypisz ją jak iloczyn czynników pierwszych z odpowiednimi wykładnikami np. dla B = 17, baza czynników = [2, 3, 5, 7, 11, 13, 17], liczba 3981970 jest 17 - gładka bo 39819780 = (2^2)\*(3^2)\*5\*7\*11\*(13^2)\*17

In [2]:
n = 39819780

# Jeśli liczba jest B-gładka, zwraca listę zawierającą wykładniki
# w kolejnści zgodnej z liczbami w bazie czynników
# np. Dla b_smooth(39819780, base=[2, 3, 5, 7, 11, 13, 17]) zwróci [2, 2, 1, 1, 1, 2, 1])

# Przykładowe rozwiązanie

def pow_idx(n, p): # Wykładnik danego czynnika pierwszego
    p_idx = 0
    while n % p == 0:
        p_idx += 1
        n /= p
    return int(n), p_idx

def preliminary_check(n, base): # Sprawdzenie czy czynniki n są w bazie czynników pierwszych
    p_indexes = [0]*len(base)

    for i, p in enumerate(base):
        n, p_indexes[i] = pow_idx(n, p)
    
    return n, p_indexes

def b_smooth(z, base): # Sprawdzanie czy liczba jest B-gładka (Tak -> zwraca wykładniki czynników pierw.)
    z, p_indexes = preliminary_check(z, base)
    
    if z != 1:
        return False
    return p_indexes

fbase = factor_base(17)

print(b_smooth(n, fbase))


[2, 2, 1, 1, 1, 2, 1]


## Zadanie 3. Relacje kongruencji z i z+n
Utwórz układ kongruencji, wyszukując pary (z, z+n) spełniające warunki opisane w materiale dydaktycznym (jako wystarczającą liczbę par przyjmij długość bazy czynników przemnożoną przez 2). Sprawdź czy za pomocą danego układu kongruencji da się otrzymać kongruencję kwadratową postaci x^2 $\equiv$ y^2 (mod n), mnożąc lub dzieląc stronami poszczególne kongruencje (czyli odpowiednio dodając lub odejmując wykładniki).

In [3]:
n = 187
B = 7

# Kongruencje są opisywane za pomocą wykładników czynników pierwszych 
# zgodnie z kolejnością w bazie czyli w formie [b_smooth(z, fbase), b_smooth(z+n, fbase)]
# gdzie lewą stroną kongruencji jest b_smooth(z, fbase), a prawą b_smooth(z+n, fbase)

# Przykładowe rozwiązanie

def create_relations(n, B): # Wyszukiwanie par z i z+n
    relations = []  # Przechowuje wykładniki czynników pierwszych
    fbase = factor_base(B)

    for z in range(2, int(n)):
        z_pow = b_smooth(z, fbase)
        z_n_pow = b_smooth(z+n, fbase)

        if z_pow and z_n_pow: # z i z+n są b-gładkie
            relations.append([z_pow, z_n_pow])

        if len(relations) >= 2*len(fbase):
            break
    
    return relations

def sum_list(list_a, list_b): # Dodawanie elementów 2 list
    return [a + b for a,b in zip(list_a, list_b)]

def even(pow_idxes1, pow_idxes2): # Sprawdza czy wykładniki po odejmowaniu są parzyste
    L = sum_list(pow_idxes1[0], pow_idxes2[0])
    P = sum_list(pow_idxes1[1], pow_idxes2[1])
    for l, p in zip(L, P):
        if (l-p) % 2 != 0:
            return False
    
    return [L, P]

relations = create_relations(n, B)

for i, r1 in enumerate(relations):
    for r2 in relations[i+1:]:
        quadratic = even(r1, r2)
        
        if quadratic:
            print(f"Otrzymanie kongruencji kwadratowej z równań {r1} i {r2} jest możliwe")

Otrzymanie kongruencji kwadratowej z równań [[1, 0, 0, 0], [0, 3, 0, 1]] i [[3, 0, 0, 1], [0, 5, 0, 0]] jest możliwe
Otrzymanie kongruencji kwadratowej z równań [[0, 2, 0, 1], [1, 0, 3, 0]] i [[7, 0, 0, 0], [0, 2, 1, 1]] jest możliwe


## Zadanie 4. Metoda Sita Liczbowego
Korzystając z wcześniej napisanych funkcji jak i materiału dydaktycznego napisz kod, który podane niżej liczby zfaktoryzuje metodą sita liczbowego.  

In [4]:
'''
Testy:
23423454 = 2^1 * 3^2 * 569^1 * 2287^1
45234523423 = 17^1 * 43^1 * 3559^1 * 17387^1
5523452342346 = 2^1 * 3^1 * 29^1 * 23003^1 * 1379993^1
7523452342312 = 2^3 * 17^1 * 281^1 * 196866557^1
8523452343241 = 11^1 * 1069^1 * 724844999^1
'''

# Przykładowe rozwiązanie

def get_factors(L, P, n, base):
    # Dzięki funkcji even() wiemy, że wykładniki nieparzyste możemy odjąć i otrzymać parzyste
    evenL = []  
    evenP = []

    for l, p in zip(L, P):
        if p%2 == 1:  # Skoro wykładnik po prawej jest nieparzysty to po lewej też musi^
            if l >= p:
                evenL.append(l-p)
                evenP.append(0)
            else:
                evenP.append(p-l)
                evenL.append(0)
        else:
            evenL.append(l)
            evenP.append(p)
    
    l_number = 1
    p_number = 1

    for i, prime in enumerate(base):
        l_number *= pow(prime, evenL[i]/2)  # Dzielenie przez 2 aby otrzymac x, a nie x^2 w kongruencji kwad.
        p_number *= pow(prime, evenP[i]/2)
    
    factor1 = gcd(abs(l_number - p_number), n)
    factor2 = gcd(l_number + p_number, n)
    
    if factor1 == 1 or factor2 == 1: # Faktoryzacja trywialna
        return False
    
    new_factors = []
    
    if not check_prime(factor1): # Sprawdanie czy czynnik jest l. złożoną
        new_factors += rational_sieve(int(factor1), base[-1], base)
    else:
        new_factors.append(factor1)
    
    if not check_prime(factor2):
        new_factors += rational_sieve(int(factor2), base[-1], base)
    else:
        new_factors.append(factor2)
    
    return new_factors


def combine_relations(n, relations, base): # Tworzenie kongruencji kwadratowej

    for i, r1 in enumerate(relations):
        for r2 in relations[i:]:
            quadratic = even(r1, r2)
            
            if quadratic:
                factors = get_factors(quadratic[0], quadratic[1], n, base)
                if factors:
                    return factors
    
    return False

def rational_sieve(n, B, primeBase=[]):

    if primeBase == []:
        primeBase = factor_base(B)

    n, factors_idxs = preliminary_check(n, primeBase)
    factors = [[primeBase[i], factors_idxs[i]] for i in range(len(factors_idxs)) if factors_idxs[i]]

    while n > 1:
        if check_prime(n):
            factors.append([n, 1])
            break

        relations = create_relations(n, B)
        new_factors = combine_relations(n, relations, primeBase)

        if not new_factors:
            print("Faktoryzacja danej liczby się nie powiodła, liczba postaci p^m\
 lub źle dobrany parameter B")
            return False
        
        else:
            for f in new_factors:
                if isinstance(f, list):
                    factors.append(f) # W przypadku gdy NWD zwróciło l. zlożoną to zwrócona została lista (rational_sieve() dla dzielnika)
                    n /= pow(f[0], f[1])
                else:
                    n, f_idx = pow_idx(n, f)
                    factors.append([int(f), f_idx])
    
    return factors


# W przypadku zmiany testów, sprawdzić sys.maxsize dla maksymalnej możliwej liczby obsługiwanej przez Python na danym komputerze
tests = [(23423454, 1000), (45234523423, 3600), 
(5523452342346, 30000), (7523452342312, 30000), (8523452343241, 30000)]
# Poprawność testów sprawdzona z Wolframalpha

base30000 = factor_base(30000)
for t in tests:
    if t[1] == 30000:
        factors = rational_sieve(t[0], t[1], base30000)
    else:
        factors = rational_sieve(t[0], t[1])
        
    if factors:
        result = f"{t[0]} = "
        for f in factors:
            result += f"{f[0]}^{f[1]} * "
        
        print(result.rstrip(" *"))


23423454 = 2^1 * 3^2 * 569^1 * 2287^1
45234523423 = 17^1 * 43^1 * 3559^1 * 17387^1
5523452342346 = 2^1 * 3^1 * 29^1 * 23003^1 * 1379993^1
7523452342312 = 2^3 * 17^1 * 281^1 * 196866557^1
8523452343241 = 11^1 * 1069^1 * 724844999^1
